In [2]:
cd /Users/amyvandiver/Library/CloudStorage/Box-Box/Nanopore/Timp_data/

/Users/amyvandiver/Library/CloudStorage/Box-Box/Nanopore/Timp_data


In [2]:
#create rotated genome 
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna

ImportError: Bio.Alphabet has been removed from Biopython. In many cases, the alphabet can simply be ignored and removed from scripts. In a few cases, you may need to specify the ``molecule_type`` as an annotation on a SeqRecord for your script to work correctly. Please see https://biopython.org/wiki/Alphabet for more information.

In [5]:
#index reference 
!samtools faidx /Users/amyvandiver/Box/Nanopore/TimpLab/Ref/GRCh38_latest_genomic.fna NC_012920.1>/Users/amyvandiver/Box/Nanopore/TimpLab/Ref/chrM.hg38.fa


In [11]:
#load in ChrM, rename to work with later code
hg38_chrm=SeqIO.read("/Users/amyvandiver/Box/Nanopore/TimpLab/Ref/chrM.hg38.fa", "fasta")

hg38_chrm.id="chrM"
hg38_chrm.name="chrM"
hg38_chrm.description="chrM"

outfile=open("chrM.hg38.fa", "w")
SeqIO.write(hg38_chrm, outfile, "fasta")
outfile.close()

In [10]:
#make rotated chrM reference by taking of first 1547 bases and appending to the end 
chrm_str=str(hg38_chrm.seq)
rot_seq=chrm_str[1547:]+chrm_str[0:1547]

obj_rot_seq=Seq(rot_seq)

hg38_rot=hg38_chrm

hg38_rot.seq=obj_rot_seq
hg38_rot.id="chrM_rot"
hg38_rot.name="chrM_rot"


outfile=open("chrM_rot.fa", "w")
SeqIO.write(hg38_rot, outfile, "fasta")
outfile.close()

In [6]:
#Set paths for GM12878 cell nanopore data

x= 'GM12878/190115_cas9enrich_mito1_multif5.gup5'
fq = x + '.fq'
ref_rot="chrM_rot.fa"
bamname=x+".rotM.bam"


In [12]:
#how many reads in run?
!echo $(( $(wc -l < $fq) / 4 )) reads

193043 reads


In [13]:
#define alingment function
def align(fastq, ref,outname):
    #align
    !minimap2 -ax map-ont {ref} -t 11 {fastq} --MD| samtools view -uF 4 | samtools sort -o {outname}
    !samtools index {outname}


In [14]:
#align to rotated reference
bamname=x+".rotM.bam"
print(bamname)
align(fq, ref_rot,bamname)

GM12878/190115_cas9enrich_mito1_multif5.gup5.rotM.bam
[M::mm_idx_gen::0.013*0.51] collected minimizers
[M::mm_idx_gen::0.016*1.51] sorted minimizers
[M::main::0.016*1.51] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.016*1.50] mid_occ = 2
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.016*1.50] distinct minimizers: 3103 (100.00% are singletons); average occurrences: 1.000; average spacing: 5.340
[M::worker_pipeline::52.500*7.30] mapped 43004 sequences
[M::worker_pipeline::76.369*7.52] mapped 42735 sequences
[M::worker_pipeline::99.919*7.64] mapped 44113 sequences
[M::worker_pipeline::119.032*7.17] mapped 44378 sequences
[M::worker_pipeline::120.746*7.08] mapped 18813 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -ax map-ont -t 11 --MD chrM_rot.fa GM12878/190115_cas9enrich_mito1_multif5.gup5.fq
[M::main] Real time: 120.750 sec; CPU: 854.631 sec; Peak RSS: 2.484 GB
[bam_sort_core] merging from 5 files...


In [15]:
#how many primary alignments?
!samtools view -F 260 -F 2048 {bamname} chrM_rot -c
!samtools view -F 260 -F 2048 {bamname} -c


142373
142373


In [ ]:
#subset forward and reverse reads for plots 
!samtools view -F 16 -F 4 -h -b {bamrot} chrM_rot > forward_rot.bam
!samtools view -F 16 -F 4 -h -b {bamname} chrM > forward.bam



In [7]:
import pandas as pd


In [8]:
#summarize chrM reads 
!samtools view -F 260 -F 2048 GM12878/190115_cas9enrich_mito1_multif5.gup5.rotM.bam chrM_rot| awk '{print length($10)}' >lengths.txt
lengths=pd.read_csv('lengths.txt',sep='\t',names=["length"])
print("Number mito reads: " + str(len(lengths)))
print("Mito reads greater than 15kb: " + str(lengths.length[lengths.length >15000].count()))
print("Mean mito read length: " + str(lengths.length.mean()))
print("Median mito read length: " + str(lengths.length.median()))


Number mito reads: 141928
Mito reads greater than 9kb: 93916
Mean mito read length: 12603.600558029424
Mean mito read length: 16458.0
Number mito reads: 142373
Mito reads greater than 9kb: 94135
Mean mito read length: 12628.419447507604
Mean mito read length: 16458.0


In [3]:
#Now call variants on nanopore data with bcftools, define function

outdir="bcftools/"
def np_bcf(bam, ref, outdir, label, chrom, af=0.05):
    chrM_reads=!samtools idxstats {bam} | grep {chrom} | cut -f3
    cov_thresh=int(chrM_reads[0])/10
    
    bcfcall=outdir+label+".bcfcall.vcf"
    
    !bcftools mpileup -a "AD,ADF,ADR" -B -Q 0 -d 100000 --threads 8 -Ou -f {ref} -r {chrom} {bam} | \
        bcftools call --ploidy 1 -Ou -m -v -P 0 | bcftools filter -e 'INFO/DP[*] < {cov_thresh}' -o {bcfcall}
    

    
    
    

In [98]:
#call variants 
np_bcf(bamrot,ref_rot,outdir,"190115_cas9enrich_chrM_rot","chrM_rot")


[mpileup] 1 samples in 1 input files


In [ ]:
#now illumina data... first converted back to fastq to align to rotated reference to be confident in comparing variant calls 

!samtools bam2fq illumina/HG001_hs38d1_300x_chrMonly_sorted.bam > illumina/HG001_ChrMonly.bam2fq.fastq

In [ ]:
#!bwa index {ref_rot}

In [ ]:
#re-align to rotated reference 
!bwa mem {ref_rot} illumina/HG001_ChrMonly.bam2fq.fastq| samtools view -bS -uF 4 | samtools sort -o HG001_ChrM_rot.bam
!samtools index HG001_ChrM_rot.bam

In [ ]:
#call variants with freebayes
!freebayes -0 -f {ref_rot} -X -F 0.05 -p 1 --pooled-continuous HG001_ChrM_rot.bam >fb/HG001_ChrM_rot.calls.vcf


In [ ]:
#filter out strand specific with vcffilter 
!vcffilter -f "SAF / AO > 0.1 & SAF / AO < 0.9 " fb/HG001_ChrM_rot.calls.vcf >fb/HG001_ChrM_rot.calls.filt.vcf

In [ ]:
from pysam import VariantFile

In [99]:
#need to convert coordinates back to normal chrM reference for publishable output, first nanopore
var=VariantFile("bcftools/190115_cas9enrich_chrM_rot.bcfcall.vcf")

newvar=VariantFile("bcftools/190115_chrM_unrot.vcf", "w", header=var.header)

for rec in var.fetch():
    newpos=rec.pos+1547
    if (newpos < 16570):
        newrec=rec.copy()
        newrec.pos=newpos
        newvar.write(newrec)
    elif (newpos>16569):
        newrec=rec.copy()
        newrec.pos=newpos-16569
        newvar.write(newrec)
        
newvar.close()
var.close()

In [72]:
#convert illumina called variants back to non-rotated reference
var=VariantFile("fb/HG001_ChrM_rot.calls.filt.vcf")
newvar=VariantFile("fb/HG001_ChrM_unrot.calls.filt.vcf", "w", header=var.header)

for rec in var.fetch():
    newpos=rec.pos+1547
    if (newpos < 16570):
        newrec=rec.copy()
        newrec.pos=newpos
        newvar.write(newrec)
    elif (newpos>16569):
        newrec=rec.copy()
        newrec.pos=newpos-16569
        newvar.write(newrec)
        
newvar.close()
var.close()